In [21]:
#librerias
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import layers
import keras_tuner as kt

# Suprimir todos los warnings
import warnings
warnings.filterwarnings('ignore')

# Configurar la semilla para la reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
import tensorflow as tf

# Listar los dispositivos físicos disponibles, en este caso, GPUs
physical_devices = tf.config.list_physical_devices('GPU')

# Si hay alguna GPU disponible
if len(physical_devices) > 0:
    # Permitir que TensorFlow crezca dinámicamente la memoria utilizada en la GPU
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Cargar el dataset

#metadata_path = '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/dataset_16_20_emb.csv'
metadata = pd.read_csv('/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/dataset_16_20_emb.csv')  # Usar la primera columna como índice


# Ruta a las imágenes
image_path = "/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/image"


# Mostrar información básica del dataset
print (metadata.head())
print(metadata['target'].value_counts())

              image_id      sex  age_approx anatom_site_general  target
0  16_ISIC_0000001.jpg  Unknown          -1             Unknown       0
1  16_ISIC_0000002.jpg  Unknown          -1             Unknown       1
2  16_ISIC_0000004.jpg  Unknown          -1             Unknown       1
3  16_ISIC_0000006.jpg  Unknown          -1             Unknown       0
4  16_ISIC_0000007.jpg  Unknown          -1             Unknown       0
target
0    64604
1     6766
Name: count, dtype: int64


In [4]:
metadata = metadata [['image_id', 'target']]
metadata

,image_id,target
0,16_ISIC_0000001.jpg,0
1,16_ISIC_0000002.jpg,1
2,16_ISIC_0000004.jpg,1
3,16_ISIC_0000006.jpg,0
4,16_ISIC_0000007.jpg,0
...,...,...
71365,20_ISIC_9999134.jpg,0
71366,20_ISIC_9999320.jpg,0
71367,20_ISIC_9999515.jpg,0
71368,20_ISIC_9999666.jpg,0


In [5]:

# Cargar el dataset
df = metadata

# Dividir el DataFrame en dos partes según el valor del target
df_majority = df[df['target'] == 0]
df_minority = df[df['target'] == 1]

# Reducir el número de muestras de la clase mayoritaria a 15,000
df_majority_downsampled = df_majority.sample(n=15000, random_state=42)

# Combinar los DataFrames balanceados
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Opcional: Mezclar el DataFrame balanceado para asegurar aleatoriedad
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Mostrar la distribución de clases después del balanceo
print(df_balanced['target'].value_counts())

df_balanced.info()

target
0    15000
1     6766
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21766 entries, 0 to 21765
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image_id  21766 non-null  object
 1   target    21766 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 340.2+ KB


In [6]:
metadata= df_balanced
metadata

,image_id,target
0,20_ISIC_6863520.jpg,0
1,20_ISIC_7746989.jpg,0
2,18_ISIC_0028930.jpg,1
3,19_ISIC_0071321.jpg,1
4,19_ISIC_0053748.jpg,0
...,...,...
21761,20_ISIC_4369062.jpg,0
21762,20_ISIC_6931277.jpg,1
21763,20_ISIC_5667935.jpg,0
21764,20_ISIC_5310630.jpg,0


In [7]:

# Dividir los datos en entrenamiento y validación
train_df, val_df = train_test_split(metadata, test_size=0.2, stratify=metadata['target'], random_state=42)

print (train_df.shape)
print (val_df.shape)

(17412, 2)
(4354, 2)


In [8]:
# Convertir los valores de la columna target a cadenas de texto
train_df['target'] = train_df['target'].astype(str)
val_df['target'] = val_df['target'].astype(str)

# Definir un ImageDataGenerator para augmentación de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Crear los generadores de datos
train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=image_path,
    x_col='image_id',
    y_col='target',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=image_path,
    x_col='image_id',
    y_col='target',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 17412 validated image filenames belonging to 2 classes.
Found 4354 validated image filenames belonging to 2 classes.


In [9]:

# Definir el modelo de Deep Learning
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Resumen del modelo
model.summary()

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 246016)            0         
                                                                 
 dense (Dense)               (None, 128)               3

2024-08-25 21:40:56.998052: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-08-25 21:40:56.998090: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-25 21:40:56.998097: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-25 21:40:56.998139: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-25 21:40:56.998159: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
# Definir el callback de Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=[early_stop]
)

Epoch 1/50


2024-08-25 21:26:51.554467: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


545/545 [==============================] - ETA: 0s - loss: 2.0150 - accuracy: 0.6504

2024-08-25 21:28:47.221633: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


545/545 [==============================] - 120s 218ms/step - loss: 2.0150 - accuracy: 0.6504 - val_loss: 0.6122 - val_accuracy: 0.7074
Epoch 2/50
545/545 [==============================] - 120s 219ms/step - loss: 0.6191 - accuracy: 0.7046 - val_loss: 0.5746 - val_accuracy: 0.7317
Epoch 3/50
545/545 [==============================] - 119s 218ms/step - loss: 0.9295 - accuracy: 0.7001 - val_loss: 0.9402 - val_accuracy: 0.6897
Epoch 4/50
104/545 [====>.........................] - ETA: 1:33 - loss: 1.2396 - accuracy: 0.5724

KeyboardInterrupt: 

In [ ]:
# Evaluar el modelo en el conjunto de validación
val_gen.reset()
predictions = model.predict(val_gen)
predictions = (predictions > 0.5).astype(int)

# Calcular el reporte de clasificación
print(classification_report(val_df['target'].astype(int), predictions))

# Guardar el modelo entrenado
model.save('skin_cancer_model.h5')

  6/137 [>.............................] - ETA: 3s

2024-08-25 21:33:16.866085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


137/137 [==============================] - 3s 24ms/step
              precision    recall  f1-score   support

           0       0.69      1.00      0.82      3001
           1       0.72      0.01      0.02      1353

    accuracy                           0.69      4354
   macro avg       0.71      0.50      0.42      4354
weighted avg       0.70      0.69      0.57      4354



In [11]:

# Definir la función del modelo
def build_model(hp):
    model = tf.keras.Sequential([
        layers.Conv2D(
            filters=hp.Int('filters', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice('kernel_size', values=[3, 5]),
            activation='relu',
            input_shape=(256, 256, 3)
        ),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(
            units=hp.Int('units', min_value=64, max_value=256, step=64),
            activation='relu'
        ),
        layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.3, max_value=0.5, step=0.1)),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# Crear el tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    hyperband_iterations=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Realizar la búsqueda
tuner.search(
    train_gen,
    epochs=10,
    validation_data=val_gen
)

# Obtener los mejores hiperparámetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

# Construir el modelo con los mejores hiperparámetros
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_gen, epochs=10, validation_data=val_gen)

Reloading Tuner from my_dir/intro_to_kt/tuner0.json

Search: Running Trial #21

Value             |Best Value So Far |Hyperparameter
64                |32                |filters
3                 |3                 |kernel_size
192               |192               |units
0.5               |0.3               |dropout_rate
0.0054677         |0.00027759        |learning_rate
4                 |10                |tuner/epochs
0                 |4                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
0                 |2                 |tuner/round



Epoch 1/4


2024-08-26 12:26:23.252484: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [12]:
# Obtener los mejores hiperparámetros encontrados hasta el momento
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

# Construir el modelo con los mejores hiperparámetros
model = tuner.hypermodel.build(best_hps)

# Entrenar el modelo con los mejores hiperparámetros
history = model.fit(train_gen, epochs=10, validation_data=val_gen)

# Evaluar y predecir con el modelo ajustado
evaluation = model.evaluate(val_gen)
print(f"Evaluation metrics: {evaluation}")

Best Hyperparameters: {'filters': 32, 'kernel_size': 3, 'units': 192, 'dropout_rate': 0.3, 'learning_rate': 0.0002775931598849941, 'tuner/epochs': 10, 'tuner/initial_epoch': 4, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0014'}
Epoch 1/10


2024-08-26 12:26:31.283457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


545/545 [==============================] - ETA: 0s - loss: 2.9951 - accuracy: 0.6380

2024-08-26 12:28:30.609664: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


545/545 [==============================] - 127s 233ms/step - loss: 2.9951 - accuracy: 0.6380 - val_loss: 0.5531 - val_accuracy: 0.7356
Epoch 2/10
545/545 [==============================] - 127s 232ms/step - loss: 0.7085 - accuracy: 0.7018 - val_loss: 0.4827 - val_accuracy: 0.7407
Epoch 3/10
545/545 [==============================] - 127s 233ms/step - loss: 0.6171 - accuracy: 0.7103 - val_loss: 0.5006 - val_accuracy: 0.7350
Epoch 4/10
545/545 [==============================] - 127s 233ms/step - loss: 0.5956 - accuracy: 0.7109 - val_loss: 0.4804 - val_accuracy: 0.7418
Epoch 5/10
545/545 [==============================] - 127s 233ms/step - loss: 1.1574 - accuracy: 0.6793 - val_loss: 1.2243 - val_accuracy: 0.6959
Epoch 6/10
545/545 [==============================] - 127s 233ms/step - loss: 0.6801 - accuracy: 0.6980 - val_loss: 0.6779 - val_accuracy: 0.7099
Epoch 7/10
545/545 [==============================] - 128s 235ms/step - loss: 0.5766 - accuracy: 0.7085 - val_loss: 0.5791 - val_accura

In [19]:
# Hacer predicciones en el conjunto de validación
y_pred = model.predict(val_gen)
y_pred = (y_pred > 0.2).astype("int32")  # Convertir probabilidades a clases binarias

# Obtener las etiquetas verdaderas del conjunto de validación
y_true = val_gen.labels

# Generar el informe de clasificación
report = classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1'])
print(report)

137/137 [==============================] - 8s 60ms/step
              precision    recall  f1-score   support

     Class 0       0.78      0.86      0.82      3001
     Class 1       0.61      0.47      0.53      1353

    accuracy                           0.74      4354
   macro avg       0.69      0.67      0.68      4354
weighted avg       0.73      0.74      0.73      4354



resultados del modelo anterior
137/137 [==============================] - 8s 60ms/step
              precision    recall  f1-score   support

     Class 0       0.78      0.86      0.82      3001
     Class 1       0.61      0.47      0.53      1353

    accuracy                           0.74      4354
   macro avg       0.69      0.67      0.68      4354
weighted avg       0.73      0.74      0.73      4354



In [20]:
# Guardar el modelo en un archivo
model.save('skin_cancer_model.h5')

aqui probamos un modelo de clasificacion con redes neuronales, como obtenemos mejores resultados, pasaremos a probar con un modelo preentrenado, el ResNet50, donde parametraremos el descongelar distintas capas a ver que resultados obtenemos.